In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import os
import requests
import tarfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch.nn.functional as F
from math import sqrt
import warnings
import xgboost as xgb
from sklearn.model_selection import train_test_split
from google.colab import files

In [ ]:
train_ids_path = "/content/drive/MyDrive/train_ids.tsv"
eval_ids_path  = "/content/drive/MyDrive/eval_ids.tsv"

def leggi_ids(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.split("\t")[0].strip() for line in f if line.strip()]

train_ids = leggi_ids(train_ids_path)
eval_ids  = leggi_ids(eval_ids_path)

participant_ids = sorted(
    [id_ for id_ in {*train_ids, *eval_ids} if id_.isdigit()],
    key=int
)

In [ ]:
base_url = "https://dcapswoz.ict.usc.edu/wwwedaic/data"
os.makedirs("estratti", exist_ok=True)

def get_target_files(pid):
    return [
        f"{pid}_P/features/{pid}_OpenFace2.1.0_Pose_gaze_AUs.csv",
        f"{pid}_P/features/{pid}_vgg16.csv",
        f"{pid}_P/features/{pid}_CNN_VGG.mat",
    ]

def extract_specific_files(tar, pid):
    target_files = get_target_files(pid)
    estratti = []

    for t in target_files:
        for member in tar.getmembers():
            tar.extract(member, path="estratti")
            estratti.append(t)
            break
    return estratti

def verify_mccl_requirements(pid, estratti):
    openface_ok = any("OpenFace2.1.0_Pose_gaze_AUs.csv" in f for f in estratti)
    f3d_ok = any("vgg16.csv" in f or "CNN_VGG.mat" in f for f in estratti)

ok_participants = []
for pid in participant_ids:
    nome_file = f"{pid}_P.tar.gz"
    link = f"{base_url}/{nome_file}"

    #print(f"\nAt... {nome_file}...")
    response = requests.get(link, stream=True)
    with open(nome_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    with tarfile.open(nome_file, "r:gz") as tar:
        estratti = extract_specific_files(tar, pid)
    if verify_mccl_requirements(pid, estratti):
        ok_participants.append(pid)

    # Cancella il file .tar.gz per non occupare spazio => Claude.AI suggestion
    os.remove(nome_file)

In [ ]:
class InterviewDataset(Dataset):
    def __init__(self, base_path, label_file, split_file, split_type='train', seq_len=1080, num_segments=3, predefined_ids=None):
        self.base_path = base_path
        self.labels = pd.read_csv(label_file)
        self.split_info = pd.read_csv(split_file)
        self.split_type = split_type
        self.seq_len = seq_len
        self.num_segments = num_segments
        self.segment_len = seq_len // num_segments
        self.data = []
        self.targets = []
        self.pids = []

        ### Chat gpt help in this to substitute rndm seed as done before
        if predefined_ids is not None:
            print(f"Usando {len(predefined_ids)} IDs predefiniti para {split_type}")
            valid_pids = [str(pid) for pid in predefined_ids]
            self.labels = self.labels[self.labels['Participant_ID'].astype(str).isin(valid_pids)]
        else:
            # usa split file normale
            split_pids = self.split_info[self.split_info['split'] == split_type]['Participant'].astype(str).tolist()
            self.labels = self.labels[self.labels['Participant_ID'].astype(str).isin(split_pids)]
        ### Chat gpt help in this to substitute rndm seed as done before

        self.labels = self.labels.rename(columns={'Participant_ID': 'pid'})
        available_ids = self._get_available_participant_ids()
        available_labels = self.labels[self.labels['pid'].astype(str).isin(available_ids)]

        print(f"{split_type} dataset: {len(available_labels)} participants available")

        for _, row in available_labels.iterrows():
            pid = str(row["pid"])
            try:
                # load openface features
                openface_file = os.path.join(base_path, f"{pid}_P/features/{pid}_OpenFace2.1.0_Pose_gaze_AUs.csv")
                df_of = pd.read_csv(openface_file)
                features_dict = {}

                f3d_cols = []
                for i in range(68):
                    f3d_cols.extend([f'x_{i}', f'y_{i}', f'z_{i}'])
                available_f3d = [col for col in f3d_cols if col in df_of.columns]
                if available_f3d:
                    features_dict['f3d'] = df_of[available_f3d].values

                gaze_cols = ['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z']
                available_gaze = [col for col in gaze_cols if col in df_of.columns]
                if available_gaze:
                    features_dict['gaze'] = df_of[available_gaze].values

                hp_cols = ['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz']
                available_hp = [col for col in hp_cols if col in df_of.columns]
                if available_hp:
                    features_dict['hp'] = df_of[available_hp].values


                fau_cols = [f'AU{i}_r' for i in [1,2,4,5,6,7,10,12,14,15,17,20,23,25,26,28,45]]
                available_faus = [col for col in fau_cols if col in df_of.columns]
                if available_faus:
                    features_dict['fau'] = df_of[available_faus].values
                min_len = min(features.shape[0] for features in features_dict.values())

                for key, features in features_dict.items():
                    features_dict[key] = features[:min_len]

                for key, features in features_dict.items():
                    features_clean = pd.DataFrame(features).dropna()
                    if len(features_clean) > 0:
                        # z-score normalization (google suggestion -> normalize each modality separately)
                        normalized = (features_clean - features_clean.mean()) / (features_clean.std() + 1e-8)
                        features_dict[key] = normalized.values
                    else:
                        continue

                processed_features = {}
                for key, features in features_dict.items():
                    if len(features) >= seq_len:
                        seq = features[:seq_len]
                    else: # padding se troppo corta

                        padding = np.zeros((seq_len - len(features), features.shape[1]))
                        seq = np.vstack((features, padding))

                    segments = []
                    for i in range(num_segments):
                        start_idx = i * self.segment_len
                        end_idx = (i + 1) * self.segment_len
                        segment = seq[start_idx:end_idx]
                        segments.append(segment)

                    processed_features[key] = np.array(segments)

                self.data.append(processed_features)
                self.targets.append(float(row["PHQ_8Total"]))
                self.pids.append(pid)

            except Exception as e:
                print(f"Error with participant {pid}: {e}")

    def _get_available_participant_ids(self):
        available_ids = []
        if os.path.exists(self.base_path):
            for item in os.listdir(self.base_path):
                if os.path.isdir(os.path.join(self.base_path, item)) and item.endswith('_P'):
                    pid = item.replace('_P', '')
                    available_ids.append(pid)
        return available_ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (
            self.data[idx],  # dict with modalities
            torch.tensor(self.targets[idx], dtype=torch.float32),
            self.pids[idx]
        )

class ResidualBlock(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.BatchNorm1d(out_dim),
            nn.ReLU(),
            nn.Linear(out_dim, out_dim),
            nn.BatchNorm1d(out_dim)
        )
        self.shortcut = nn.Linear(in_dim, out_dim) if in_dim != out_dim else nn.Identity()
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.shortcut(x)
        out = self.conv(x)
        return self.relu(out + residual)

class ModalityEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_heads=4):
        super().__init__()
        self.hidden_dim = hidden_dim

        # feature extractor (like ResNet - consiglio di ChatGPT)
        self.feature_extractor = nn.Sequential(
            ResidualBlock(input_dim, hidden_dim),
            ResidualBlock(hidden_dim, hidden_dim)
        )
        self.bilstm = nn.LSTM(
            hidden_dim, hidden_dim // 2,
            batch_first=True, bidirectional=True
        )

        self.attention = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        batch_size, seq_len, input_dim = x.shape
        x_flat = x.reshape(-1, input_dim)
        features = self.feature_extractor(x_flat)
        features = features.reshape(batch_size, seq_len, self.hidden_dim)

        # BiLSTM
        lstm_out, _ = self.bilstm(features)
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        output = self.norm(lstm_out + attn_out)

        return output

class CrossCharacteristicAttention(nn.Module):
    # attention tra modalità diverse
    def __init__(self, hidden_dim, num_heads=4):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        self.cross_attention = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.norm = nn.LayerNorm(hidden_dim)

        # gate mechanism per controllare interactions
        self.gate = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Sigmoid()
        )

    def forward(self, modality_features):
        enriched_features = {}
        modality_names = list(modality_features.keys())

        for i, mod_i in enumerate(modality_names):
            features_i = modality_features[mod_i]
            enhanced_i = features_i.clone()

            for j, mod_j in enumerate(modality_names):
                if i != j:
                    features_j = modality_features[mod_j]
                    cross_attn, _ = self.cross_attention(features_i, features_j, features_j)
                    concat_features = torch.cat([features_i, cross_attn], dim=-1)
                    gate_weights = self.gate(concat_features)
                    enhanced_i = enhanced_i + gate_weights * cross_attn

            enriched_features[mod_i] = self.norm(enhanced_i)

        return enriched_features

class TemporalSegmentAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=4):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, segment_features):
        attn_output, attn_weights = self.attention(segment_features, segment_features, segment_features)
        return self.norm(segment_features + attn_output), attn_weights

class MCCLModel(nn.Module):
    def __init__(self, modality_dims, hidden_dim=64, num_heads=4, num_segments=3):
        super().__init__()
        self.modality_dims = modality_dims
        self.hidden_dim = hidden_dim
        self.num_segments = num_segments

        # encoder cada modalidade !=
        self.modality_encoders = nn.ModuleDict({
            modality: ModalityEncoder(dim, hidden_dim, num_heads)
            for modality, dim in modality_dims.items()
        })

        # cross ch. attention
        self.cross_char_attention = CrossCharacteristicAttention(hidden_dim, num_heads)

        # temporal attention
        self.temporal_segment_attention = TemporalSegmentAttention(hidden_dim, num_heads)

        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * len(modality_dims), hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim)
        )
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, x, return_features=True):
        batch_size = next(iter(x.values())).size(0)
        segment_representations = []

        for segment_idx in range(self.num_segments):
            segment_modality_features = {}

            for modality, data in x.items():
                segment_data = data[:, segment_idx, :, :]
                encoded = self.modality_encoders[modality](segment_data)
                segment_modality_features[modality] = encoded

            enriched_features = self.cross_char_attention(segment_modality_features)

            segment_pooled = []
            for modality, features in enriched_features.items():
                pooled = features.mean(dim=1)
                segment_pooled.append(pooled)
            segment_concat = torch.cat(segment_pooled, dim=-1)
            segment_fused = self.fusion(segment_concat)
            segment_representations.append(segment_fused)

        segment_features = torch.stack(segment_representations, dim=1)
        attended_segments, attention_weights = self.temporal_segment_attention(segment_features)
        global_features = attended_segments.mean(dim=1)
        output = self.regressor(global_features).squeeze(-1)

        if return_features:
            return output, global_features, attention_weights
        else:
            return output

def contrastive_loss_paper_style(features, labels, temperature=0.07, margin=2.0):
    # normalize features
    features_norm = F.normalize(features, p=2, dim=1)

    sim_matrix = torch.mm(features_norm, features_norm.t()) / temperature
    labels_expanded = labels.unsqueeze(1)
    label_diff = torch.abs(labels_expanded - labels_expanded.t())

    # positives: same values or very similar
    pos_mask = (label_diff < margin).float()
    pos_mask.fill_diagonal_(0)  # no som con sé stesso
    neg_mask = (label_diff >= margin * 2).float()

    # InfoNCE style loss come proposto da Zh.
    pos_sim = sim_matrix * pos_mask
    neg_sim = sim_matrix * neg_mask

    pos_loss = -torch.log(torch.sigmoid(pos_sim) + 1e-8) * pos_mask
    neg_loss = -torch.log(1 - torch.sigmoid(neg_sim) + 1e-8) * neg_mask

    pos_loss = pos_loss.sum() / (pos_mask.sum() + 1e-8)
    neg_loss = neg_loss.sum() / (neg_mask.sum() + 1e-8)

    return pos_loss + neg_loss

def contrastive_pretraining(model, train_loader, epochs=10, lr=1e-3):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for batch_idx, (x, y, _) in enumerate(train_loader):
              # convert modality dict to tensors cf. Reddit
            x_tensors = {}
            for modality, data in x.items():
                x_tensors[modality] = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).to(device)
            y = y.to(device)

            optimizer.zero_grad()

            # forward pass para get features
            _, features, _ = model(x_tensors, return_features=True)
            loss = contrastive_loss_paper_style(features, y)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if batch_idx % 5 == 0:
                print(f"Pre-train Epoch {epoch+1}/{epochs} | Batch {batch_idx} | Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        print(f"Pre-train Epoch {epoch+1} completed | Avg Loss: {avg_loss:.4f}")

def fine_tuning_with_contrastive(model, train_loader, val_loader=None, epochs=20, lr=1e-4, contrastive_weight=0.1):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    best_val_mae = float('inf')

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        total_mae = 0.0
        total_rmse = 0.0

        for batch_idx, (x, y, _) in enumerate(train_loader):
            x_tensors = {}
            for modality, data in x.items():
                x_tensors[modality] = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).to(device)
            y = y.to(device)

            optimizer.zero_grad()

            output, features, _ = model(x_tensors, return_features=True)
            reg_loss = criterion(output, y)

            # contrastive + comb. loss
            cont_loss = contrastive_loss_paper_style(features, y)
            loss = reg_loss + contrastive_weight * cont_loss

            # backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # gradient clipping
            optimizer.step()

            total_loss += loss.item()
            mae = mean_absolute_error(y.cpu().numpy(), output.detach().cpu().numpy())
            rmse = sqrt(mean_squared_error(y.cpu().numpy(), output.detach().cpu().numpy()))
            total_mae += mae
            total_rmse += rmse

        scheduler.step()

        val_mae, val_rmse = 0.0, 0.0
        if val_loader is not None:
            model.eval()
            with torch.no_grad():
                for x_val, y_val, _ in val_loader:
                    x_val_tensors = {}
                    for modality, data in x_val.items():
                        x_val_tensors[modality] = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).to(device)
                    y_val = y_val.to(device)

                    output_val = model(x_val_tensors, return_features=False)
                    val_mae += mean_absolute_error(y_val.cpu().numpy(), output_val.cpu().numpy())
                    val_rmse += sqrt(mean_squared_error(y_val.cpu().numpy(), output_val.cpu().numpy()))

            val_mae /= len(val_loader)
            val_rmse /= len(val_loader)

            if val_mae < best_val_mae:
                best_val_mae = val_mae
                torch.save(model.state_dict(), 'best_mccl_model.pth')

        avg_loss = total_loss / len(train_loader)
        avg_mae = total_mae / len(train_loader)
        avg_rmse = total_rmse / len(train_loader)

def evaluate_model(model, dataloader):
    # evaluacao do modelo
    model.eval()
    predictions = []
    targets = []
    participant_ids = []

    with torch.no_grad():
        for x, y, pids in dataloader:
            x_tensors = {}
            for modality, data in x.items():
                x_tensors[modality] = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).to(device)

            output = model(x_tensors, return_features=False)

            predictions.extend(output.cpu().numpy())
            targets.extend(y.cpu().numpy())
            participant_ids.extend(pids)

    predictions = np.array(predictions)
    targets = np.array(targets)
    #SSe adicionado como previsto cf. whatsapp
    mae = mean_absolute_error(targets, predictions)
    rmse = sqrt(mean_squared_error(targets, predictions))
    sse = np.sum((targets - predictions)**2)

    return mae, rmse, sse, predictions, targets, participant_ids

def create_results_table(predictions, targets, participant_ids):
    results_df = pd.DataFrame({
        'Participant_ID': participant_ids,
        'PHQ8_Original': targets,
        'PHQ8_Predicted': predictions,
        'Absolute_Error': np.abs(targets - predictions)
    })
    results_df = results_df.sort_values('Participant_ID')

    return results_df

def extract_features_for_xgboost(model, dataloader):
    model.eval()
    features_list = []
    targets_list = []
    pids_list = []

    with torch.no_grad():
        for x, y, pids in dataloader:
            x_tensors = {}
            for modality, data in x.items():
                x_tensors[modality] = torch.stack([torch.tensor(d, dtype=torch.float32) for d in data]).to(device)

            _, features, _ = model(x_tensors, return_features=True)

            features_list.append(features.cpu().numpy())
            targets_list.extend(y.cpu().numpy())
            pids_list.extend(pids)

    features_array = np.vstack(features_list)
    targets_array = np.array(targets_list)

    return features_array, targets_array, pids_list

def train_xgboost_final(train_features, train_targets, val_features, val_targets):
          # train XGBoost with extracted features per la fine
    xgb_params = {
        'objective': 'reg:squarederror',
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'n_estimators': 200,
        'early_stopping_rounds': 20
    }

    print("Training XGBoost finale...")
    xgb_model = xgb.XGBRegressor(**xgb_params)

    xgb_model.fit(
        train_features, train_targets,
        eval_set=[(val_features, val_targets)],
        verbose=False
    )

    return xgb_model

In [ ]:
# main execution
def main():
    base_path = "/content/estratti"
    label_file = "/content/drive/MyDrive/Detailed_PHQ8_Labels.csv"
    split_file = "/content/drive/MyDrive/detailed_lables.csv"
    seq_len = 1080  # Lunghezza sequenza per E-DAIC come da paper
    num_segments = 3  # 3 segmenti temporali come nel paper
    train_ids_file = "/content/drive/MyDrive/train_ids.tsv"
    eval_ids_file = "/content/drive/MyDrive/eval_ids.tsv"

    train_ids_df = pd.read_csv(train_ids_file, sep='\t')
    eval_ids_df = pd.read_csv(eval_ids_file, sep='\t')
    train_participant_ids = train_ids_df.iloc[:, 0].astype(str).tolist()
    eval_participant_ids = eval_ids_df.iloc[:, 0].astype(str).tolist()

    try:
        train_dataset = InterviewDataset(
            base_path, label_file, split_file,
            split_type='train', seq_len=seq_len, num_segments=num_segments,
            predefined_ids=train_participant_ids
        )

        val_dataset = InterviewDataset(
            base_path, label_file, split_file,
            split_type='train', seq_len=seq_len, num_segments=num_segments,
            predefined_ids=eval_participant_ids
        )
    except Exception as e:
        print(f"Errore nel caricamento dataset: {e}")
        return

    sample_data = train_dataset[0][0]
    modality_dims = {}
    for modality, data in sample_data.items():
        modality_dims[modality] = data.shape[-1]

    train_split_dataset = train_dataset
    val_split_dataset = val_dataset
    batch_size = min(4, len(train_split_dataset))

    def collate_fn(batch):
        modality_data = {}
        targets = []
        pids = []

        for modality in batch[0][0].keys():
            modality_data[modality] = [item[0][modality] for item in batch]

        targets = [item[1] for item in batch]
        pids = [item[2] for item in batch]

        return modality_data, torch.stack(targets), pids

    train_loader = DataLoader(train_split_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_split_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = MCCLModel(
        modality_dims=modality_dims,
        hidden_dim=128,  # da 264; con 64 non cambia granché
        num_heads=4,
        num_segments=num_segments
    )

    total_params = sum(p.numel() for p in model.parameters())

    #Contrastativo
    contrastive_pretraining(
        model,
        train_loader,
        epochs=10,  # Pre-training contrastivo
        lr=1e-3
    )

    # Salva modello dopo pre-training
    torch.save(model.state_dict(), '/content/mccl_pretrained.pth')

    # Fine-tuning con Regression + Contrastive Loss
    fine_tuning_with_contrastive(
        model,
        train_loader,
        val_loader,
        #epochs=10,  # Fine-tuning
        epochs=20,  # Fine-tuning
        lr=1e-4,
        contrastive_weight=0.1
    )

    if os.path.exists('best_mccl_model.pth'):
        model.load_state_dict(torch.load('best_mccl_model.pth'))

    # Estrai features per XGBoost
    train_features, train_targets, train_pids = extract_features_for_xgboost(model, train_loader)
    val_features, val_targets, val_pids = extract_features_for_xgboost(model, val_loader)
    xgb_model = train_xgboost_final(train_features, train_targets, val_features, val_targets)
    xgb_predictions = xgb_model.predict(val_features)
    mae_xgb = mean_absolute_error(val_targets, xgb_predictions)
    rmse_xgb = sqrt(mean_squared_error(val_targets, xgb_predictions))
    sse_xgb = np.sum((val_targets - xgb_predictions)**2)
    results_df = create_results_table(xgb_predictions, val_targets, val_pids)

      # Risultati salvati qui in content => POI LI DEVI SCARICARE!!!
    results_df.to_excel('/content/mccl_results.xlsx', index=False)

    # Stampa delle metriche finali
    print("\nRisultati finali XGBoost (validazione):")
    print("MAE:", mae_xgb)
    print("RMSE:", rmse_xgb)
    print("SSE:", sse_xgb)

    # Mostra le prime 10 righe dei risultati
    print("\nFirst 10 (to see più o meno):")
    print(results_df.head(10))

    torch.save(model.state_dict(), '/content/mccl_model_final.pth')

    import pickle
    with open('/content/xgboost_final_model.pkl', 'wb') as f:
        pickle.dump(xgb_model, f)

    return mae_xgb, rmse_xgb, sse_xgb, results_df

if __name__ == "__main__":
    main()

#Qui sopra OK; ricordati di riconnetterti al runtime + fai che abbia train e val mandati da joao sul gruppo;
#i risultati sono grossomodo fedeli al paper

In [ ]:
excel_file = '/content/mccl_results.xlsx'
files.download(excel_file)